In [16]:
!pip install geopy

In [3]:
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import time

In [17]:
# CSV 파일 로드
df = pd.read_csv("metropoli.csv")
df.head()

,Unnamed: 0,출발 행정동 코드,도착 행정동 코드,출발 행정동 주소,도착 행정동 주소
0,0,1101053,1101053,서울특별시 종로구 사직동,서울특별시 종로구 사직동
1,1,1101053,1101053,서울특별시 종로구 사직동,서울특별시 종로구 사직동
2,2,1101053,1101053,서울특별시 종로구 사직동,서울특별시 종로구 사직동
3,3,1101053,1101053,서울특별시 종로구 사직동,서울특별시 종로구 사직동
4,4,1101053,1101053,서울특별시 종로구 사직동,서울특별시 종로구 사직동


In [18]:
df = df.iloc[:, 1: 5]
df.shape

(14680050, 4)

In [19]:
df.head()

,출발 행정동 코드,도착 행정동 코드,출발 행정동 주소,도착 행정동 주소
0,1101053,1101053,서울특별시 종로구 사직동,서울특별시 종로구 사직동
1,1101053,1101053,서울특별시 종로구 사직동,서울특별시 종로구 사직동
2,1101053,1101053,서울특별시 종로구 사직동,서울특별시 종로구 사직동
3,1101053,1101053,서울특별시 종로구 사직동,서울특별시 종로구 사직동
4,1101053,1101053,서울특별시 종로구 사직동,서울특별시 종로구 사직동


In [27]:
# 1. Geocoder setup
geolocator = Nominatim(user_agent="metro_coder")  # change app name if needed
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

In [29]:
# 2. Caching dictionary
geo_cache = {}

In [31]:
# 3. Function to get lat/lon with caching
def get_lat_lon_cached(address):
    if pd.isna(address):
        return pd.Series([None, None])
    
    if address in geo_cache:
        return geo_cache[address]
    
    try:
        location = geocode(address)
        if location:
            result = pd.Series([location.latitude, location.longitude])
        else:
            result = pd.Series([None, None])
    except Exception as e:
        print(f"Error for {address}: {e}")
        result = pd.Series([None, None])
    
    geo_cache[address] = result
    return result



In [27]:
# 5. Apply geocoding
df[['출발_lat', '출발_lon']] = df['출발 행정동 주소'].apply(get_lat_lon_cached)

# 6. View result
print(df.head())

          출발행정코드      출발도   출발시  출발동             출발 주소      도착행정코드      도착도  \
0     5013059000  제주특별자치도  서귀포시  대륜동  제주특별자치도 서귀포시 대륜동  5013054000  제주특별자치도   
1     5013054000  제주특별자치도  서귀포시  천지동  제주특별자치도 서귀포시 천지동  5013057000  제주특별자치도   
2     5013057000  제주특별자치도  서귀포시  동홍동  제주특별자치도 서귀포시 동홍동  5013054000  제주특별자치도   
3     5013054000  제주특별자치도  서귀포시  천지동  제주특별자치도 서귀포시 천지동  5013059000  제주특별자치도   
4     5011062000  제주특별자치도   제주시  봉개동   제주특별자치도 제주시 봉개동  5011059000  제주특별자치도   
...          ...      ...   ...  ...               ...         ...      ...   
4987  5011067000  제주특별자치도   제주시  외도동   제주특별자치도 제주시 외도동  5011069000  제주특별자치도   
4988  5013055000  제주특별자치도  서귀포시  효돈동  제주특별자치도 서귀포시 효돈동  5013025300  제주특별자치도   
4989  5013025300  제주특별자치도  서귀포시  남원읍  제주특별자치도 서귀포시 남원읍  5013055000  제주특별자치도   
4990  5011066000  제주특별자치도   제주시  노형동   제주특별자치도 제주시 노형동  5011069000  제주특별자치도   
4991  5011069000  제주특별자치도   제주시  도두동   제주특별자치도 제주시 도두동  5011066000  제주특별자치도   

       도착시  도착동             도착 주소  start_lat   star

In [ ]:
df[['도착_lat', '도착_lon']] = df['도착 행정동 주소'].apply(get_lat_lon_cached)

In [ ]:
# 출발/도착 위경도
df[['start_lat', 'start_lon']] = df['출발 주소'].apply(get_lat_lon)
df[['end_lat', 'end_lon']] = df['도착 주소'].apply(get_lat_lon)

# 중심 좌표
df['중심_lat'] = (df['start_lat'] + df['end_lat']) / 2
df['중심_lon'] = (df['start_lon'] + df['end_lon']) / 2

print(df['중심_lat'])
print(df['중심_lon'])